# **Logit Lens: практика**

Привет, друзья! Добро пожаловать на практику по Logit Lens. В этой работе вы шаг за шагом реализуете логит-линзу — начиная, как всегда, от анализа архитектуры и заканчивая построением результатов.

Во время выполнения практики, вы:

* Проанилизруете ахитектуру GPT-2;
* Соберете логиты с каждой необходимой компоненты модели;
* Построите визуализацию полученного результата;
* Познакомитесь с билиотекой NNsight для упрощения построения логит-линз;


[Оригинальная статья о LogitLiens](https://www.lesswrong.com/posts/AcKRB8wDpdaN6v6ru/interpreting-gpt-the-logit-lens)


Приступим!

# **Часть 1. Logit Lens своими руками**

In [ ]:
!pip install torchinfo==1.8.0 -q

In [ ]:
# Import libraries
from IPython.display import clear_output
from typing import List, Callable
import torch
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from torchinfo import summary
import plotly.express as px
import plotly.io as pio


clear_output()

Начнем с загрузки модели.

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.eval();

**Задание 1. Вызовите атрибут ._modules у модели. Сколько модулей верхнего уровня вы получили?**

In [ ]:
# Ваш код здесь

Рассмотрим процесс преобразования входного примера в GPT-2 step by step. Для этого напишем простой `input` и проанилизуем информацию о модели

In [ ]:
text = "Are cats good?"
encoded_input = tokenizer(text, return_tensors='pt')

input0 = torch.tensor(
    [[tokenizer.encode(text, add_special_tokens=True)]]
)
outputs = model(input0)

summary(model, input_data=input0)

**Задание 2-4. Изучите карточку, полученную выше. Ответьте на вопросы:**

**1. Сколько всего параметров в модели?** 163,037,184 \
**2. Сколько в модели блоков трансфомера?** 12 \
**3. Сколько в модели выходных значений (токенов)?** 50257 \

Отлично, вы примерно посмотрели на модель. Теперь посмотрим, как она работает — пропустим input через модель, с целью получить не только информацию о скрытых состояниях и выходны логитах, но и генерацию какого-то текста.

In [ ]:
output_ids = model.generate(**encoded_input, max_length=14, min_length=14, return_dict_in_generate=True, output_hidden_states=True)

# Декодирование
output_text = tokenizer.decode(output_ids[0][0], skip_special_tokens=True)
output_text

**Обратите внимание:** первые выходные токены генерации — в точности равны input'у.

**Задание 5. Сравните `output_ids['sequences']` и `encoded_input['input_ids']`. Сколько следующих за входными токенов сгенерировала модель?**

In [ ]:
# Ваш код здесь

In [ ]:
# Ваш код здесь

**Задание 6. Сколько токенов сгенерировала модель всего?**



In [ ]:
# Ваш код здесь

Теперь рассмотрим скрытые состояния. Всего их в значении полученного словаря 10. Однако, блоков трансформера в модели 12 и, кроме того, скрытое состояние фиксируется после слоя `embedding`.

**Почему для нашего входа мы получили 10 скрытых состояний:** \

Для GPT моделей возвращаются `hidden states` каждого сгенерированного токена и для них идёт детализация `hidden states` модели. То есть, так как у нас было сгенерировано 10 новых токенов, то у нас получается по 13 скрытых состояний на каждый токен.  



In [ ]:
hs_tokens_cnt = len(output_ids['hidden_states'])
hs_f_each_tokens_cnt = len(output_ids['hidden_states'][0])

print(f'Количество токенов, для которых сгенерированы скрытые состояния {hs_tokens_cnt}\nКоличество скрытых состояний на токен: {hs_f_each_tokens_cnt}')

**Ещё раз закрепим:**

Для каждого сгенерированного токена, 13 скрытых состояний. Вы можете убедиться в этом, позапускав код ниже с разными токенами (менять только первую строку).

Кстати, обратите внимание, что для токена 198 (значение `\n`) у нас только один набор скрытых состояний. Отсюда, имеем:

* Набор скрытых состояний 0 для входных значений: `tensor([ 8491, 11875,   922,    30])`
* Набор скрытых состояний  1 для токена: `tensor([198])`
* Набор скрытых состояний 2 для токена: `tensor([464])`
* Набор скрытых состояний 3 для токена: `tensor([3280])`
* Набор скрытых состояний 4 для токена: `tensor([318])`
* Набор скрытых состояний 5 для токена: `tensor([3763])`
* Набор скрытых состояний 6 для токена: `tensor([13])`
* Набор скрытых состояний 7 для токена: `tensor([28997])`
* Набор скрытых состояний 8 для токена: `tensor([389])`
* Набор скрытых состояний 9 для токена: `tensor([922])`



In [ ]:
# Extract hidden states (last layer)
hidden_states = output_ids.hidden_states[0]  # Скрытые состояния i-го токена, строка, в которой можно перебирать номера

last_hidden_state = hidden_states[0]  # Последнее состояние i-го токена

# Извлекаем последние слои для проекции на словарь
lm_head = model.lm_head  # linear layer to dict

# Преобразуем hidden states в logits вручную
logits_from_hidden_states = lm_head(last_hidden_state)  # (batch_size, seq_length, vocab_size)

# Проверяем корректность размерность
print(f"Logits shape: {logits_from_hidden_states.shape}")  # Should be (batch_size, seq_length, vocab_size)

# Конвертируем logits в предикты (argmax по vocab size)
predicted_token_ids = torch.argmax(logits_from_hidden_states, dim=-1)

# Декодируем predicted tokens
decoded_predictions = tokenizer.batch_decode(predicted_token_ids, skip_special_tokens=True)

# Результаты
print(f"Predicted tokens:\n{decoded_predictions}")

Теперь, поняв, что мы собираем для каждого токена мы можем собрать проекции по всем токенам.

In [ ]:
lens_dict = {}
logits_dict = {}

for token_idx, token_hidden_states in enumerate(output_ids.hidden_states): # для кортежа скрытых состояний каждого токена
  lens_dict[token_idx] = []
  logits_dict[token_idx] = []

  for hidden_idx, hidden in enumerate(token_hidden_states): # для кадого скрытого состояния внутри

    logits = lm_head(hidden) # извлекаем логиты

    probs = torch.nn.functional.softmax(logits, dim=-1) # извлекаем вероятности
    predicted =  torch.argmax(probs, dim=-1) # извлекаем номер спрогнозированного токена

    proba =  torch.max(probs, dim=-1).values # извлекаем вероятность спрогнозированного токена
    logits_dict[token_idx].append(proba)

    decoded = tokenizer.batch_decode(predicted, skip_special_tokens=True)
    lens_dict[token_idx].append(decoded[0])

**Задание 7. Сколько всего проекций у вас получилось?**

In [ ]:
# Ваш код здесь для каждого токена (их 10) 13 скрытых состояний = 10*13

Теперь, прежде чем визуализировать результат, удалим первые токены и логиты, так как они отражают input. Кроме того, первый выход содержит несколько токенов, что не удобно для отображения на графике.

In [ ]:
logits_dict.pop(0)
lens_dict.pop(0)

### **Визуализация результата**

Перейдем к визуализации. Все собранные логиты и токены можно рассмотреть как матрицы, где строки — это скрытых слоев, а столбцы — токены.

* Первая строка — представление сгенерированных токенов на 1 скрытом слое; \
* Вторая строка — представление сгенерированных токенов на 2 скрытом слое;
* ...и так далее...
* Последняя строка — представление сгенерированных токенов на 13м скрытом слое (посленднем энкодере модели).

Матрицу, в свою очередь, можно визуилизровать как тепловую карту. Это реализовано ниже шаг за шагом.

In [ ]:
# Создадим матрицу для вероятностей
probs_matrix = []
tokens_matrix = []

# Перебираем все токены
for token_idx in range(1, len(lens_dict)+1):
    probs_matrix.append(logits_dict[token_idx])  # Собираем вероятности
    tokens_matrix.append(lens_dict[token_idx])  # Собираем токены

# Преобразуем оба словаря в numpy для удобства работы с тепловыми картами
probs_matrix = torch.stack([torch.tensor(i) for i in probs_matrix], dim=1).numpy()
tokens_matrix = np.stack([i for i in tokens_matrix])

# Визуализируем вероятности с помощью тепловой карты
plt.figure(figsize=(12, 8))
sns.heatmap(probs_matrix, cmap='YlGnBu', annot=True, fmt=".2f", cbar=True)

# Настроим метки на осях
plt.title("Logit Lens: Token Probabilities Across Layers")
plt.xlabel("Hidden States (Token Indexes)")
plt.ylabel("Layers")
plt.yticks(ticks=[i - 0.5 for i in range(1, len(lens_dict[1]) +1)], labels=[f'Layer {i}' for i in range(1, len(lens_dict[1]) + 1)], rotation=0)
plt.xticks(ticks=[i + 0.5 for i in range(0, len(lens_dict))], labels=[f"Token {i+1}" for i in range(0, len(lens_dict))])

# И покажем карту
plt.show()

С помощью `matplotlib` мы можем визуализировать в виде тепловой карты только логиты. Идеально было бы на каждую ячейку наложить вместе с логитами также спрогнозированные слова.

Это можно делать с помощью `plotly` — интерактивной (но сильно более тяжелой по памяти) библиотеки для визуализации. Ниже я также привожу код и для неё.


In [ ]:
output_token_ids = output_ids['sequences'][0][5:] # выходные токены, для записи их номеров по оси х

In [ ]:
fig = px.imshow(
    probs_matrix, # матрица вероятностьй
    x=[str(i.item()) for i in output_token_ids], # выходные токены, для записи их номеров по оси х
   # y=[f'L-r {i}' for i in list(range(len(tokens_matrix.T)))], # для отображения на оси y названий (опционально)
    color_continuous_scale=px.colors.diverging.RdYlBu_r, # смена циврофой палитры
    color_continuous_midpoint=0.50,  # центральная точка для цветовой палитры — у нас это 0.5, так как логиты мы преобразовали в вероятности
    labels=dict(x="Out Tokens", y="Layers", color="Probability") # названия для осби x, y и colorbar
)

# нанесение названия на график

fig.update_layout(
    title='Logit Lens Visualization'
)

# нанесение текста на ячейки

fig.update_traces(text=tokens_matrix.T, texttemplate="%{text}")

# визуализация фигуры

fig.show()

Зафиксируем выводы из графика:

1. Выходы логит-линзы на первом слое приближенно равны выходам логит-линзы на последнем слое со сдвигом;
2. Вероятности не имеют стабильного повышения или понижения по слоям. На средних слоях модель наиболее уверенна в токенах;

# **Часть 2. NNsight**


[NNsight](https://nnsight.net/) — библиотека, которая за счет внутренних оптимизаций, позволяет обвешивать Hf модельки так, чтобы извлекать скрытые состояния для дальнейшего анализа быстро и просто.

**Преимущества библиотеки:**

1. Скорость запуска;
2. Уобный интерфейс;
3. Плюс понятные туториалы с визуализациями.

**Некоторые недостатки:**

Как пишут авторы, библиотека находится на стадии становления. Поэтому в бибилотеке еще есть что улучшать.

1. Не все модели с Hf грузятся.
2. Читатели моего блога поделились, что при использовании могут быть ошибки с градиентами в графе.

Однако для учебы целей — библиотека великолепна! Давайте посмотрим, как построить линзу с её помощью.



In [ ]:
!pip install -U nnsight -q

В данной библиотеке модели загружаются в классы обертки при помощи синтаксика, аналогичного transformers.

In [ ]:
# Загрузка GPT2
from nnsight import LanguageModel

model_NN = LanguageModel("openai-community/gpt2", device_map="auto", dispatch=True)

Посмотрим на архитектуру модели.

In [ ]:
print(model_NN)

Архитектура почти не различается. Разве что есть компоненты `transformer`, `generator`, `streamer`. В документации они точно не описаны.

Теперь посмотрим на реализацию логит-линзы из [туториала](http://nnsight.net/notebooks/tutorials/logit_lens/) авторов библиотеки. Воспроизведем его с тем же текстом, что мы использовали для линзы, построенной вручную.

In [ ]:
prompt= "Аre cats good?"
layers = model_NN.transformer.h
probs_layers = []

with model_NN.trace() as tracer:
    with tracer.invoke(prompt) as invoker:
        for layer_idx, layer in enumerate(layers):

            # Преобразование head + layer normalization
            layer_output = model_NN.lm_head(model_NN.transformer.ln_f(layer.output[0]))

            # Применение softmax
            probs = torch.nn.functional.softmax(layer_output, dim=-1).save()
            probs_layers.append(probs)

probs = torch.cat([probs.value for probs in probs_layers])

# Забираем максимальные вероятности
max_probs, tokens = probs.max(dim=-1)

# Декодируем id в токены
words = [[model_NN.tokenizer.decode(t.cpu()).encode("unicode_escape").decode() for t in layer_tokens]
    for layer_tokens in tokens]

# 'input_ids'
input_words = [model_NN.tokenizer.decode(t) for t in invoker.inputs[0][0]["input_ids"][0]]

## **Визуализация**

In [ ]:
pio.renderers.default = "colab"


fig = px.imshow(
    max_probs.detach().cpu().numpy(),
    x=input_words,
    y=list(range(len(words))),
    color_continuous_scale=px.colors.diverging.RdYlBu_r,
    color_continuous_midpoint=0.50,
    text_auto=True,
    labels=dict(x="Input Tokens", y="Layers", color="Probability")
)

fig.update_layout(
    title='Logit Lens Visualization',
    xaxis_tickangle=0
)

fig.update_traces(text=words, texttemplate="%{text}")
fig.show()

Обратите внимание, что результаты не соотносятся. И это подсвечивает проблему в использовании оберточных библиотек — вы не полностью контроллируете результат и не можете увидеть каждую компоненту, которая к нему приводит.


Спасибо за вашу работу!